Hi there! This notebook covers the basic `ptplot` interface for making plots. You'll see how easy it is to generate an interactive, beautiful-looking plot showing player-tracking data, as well as some knobs you can tweak to adjust things just to how you want. 

_(NOTE: If you are viewing this notebook on GitHub, none of the plots will render due to the [way GitHub processes the notebooks](https://github.com/plotly/plotly.py/issues/931). To see the notebook rendered properly, please [use nbviewer](https://nbviewer.jupyter.org/github/AndrewRook/ptplot/blob/main/notebooks/1-Basic_Plots.ipynb).)_

We'll start with some simple imports.

In [1]:
import pandas as pd

from ptplot import PTPlot
from ptplot.hover import Hover
from ptplot.nfl import Aesthetics, Field
from ptplot.plot import Positions, Tracks

from bokeh.plotting import show
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

We'll pick a single play to look at, just for demonstration purposes. I've somewhat-randomly chosen [this Cleveland Browns reverse + WR pass from the 2018 season](https://www.youtube.com/watch?v=Co5IaAFdaPU), which has player-tracking data available for it from Adam Sonty's [ngs_highlights repo](https://github.com/asonty/ngs_highlights) (specifically, the file used is located [here](https://raw.githubusercontent.com/asonty/ngs_highlights/20f74138f0a2f7e91879d31bbf2d8addc0bd59bc/play_data/2018_CLE_2018122305_1246.tsv)).

In [2]:
# Data file copied locally to ensure the demo works even if the ngs_highlights_repo is taken down
player_tracking_data = pd.read_csv(
    "2018_CLE_2018122305_1246.tsv",
    sep="\t", parse_dates=["time"]
)
player_tracking_data.loc[player_tracking_data["displayName"] == "ball", "teamAbbr"] = "ball"
player_tracking_data.loc[player_tracking_data["displayName"] == "ball", "jerseyNumber"] = ""

In [3]:
player_tracking_data.head().T

,0,1,2,3,4
gameId,2018122305,2018122305,2018122305,2018122305,2018122305
playId,1246,1246,1246,1246,1246
playType,play_type_pass,play_type_pass,play_type_pass,play_type_pass,play_type_pass
season,2018,2018,2018,2018,2018
seasonType,REG,REG,REG,REG,REG
week,16,16,16,16,16
preSnapHomeScore,7,7,7,7,7
preSnapVisitorScore,0,0,0,0,0
playDirection,right,right,right,right,right
quarter,2,2,2,2,2


Let's start by making a simple plot that shows player positions at a certain frame. Frames where interesting stuff happens are actually tagged by the event column, so we'll choose the moment of time when the pass occurred:

In [4]:
plot = (
    PTPlot(data=player_tracking_data[player_tracking_data["event"] == "pass_forward"], pixel_height=400) 
    + Field() 
    + Positions(
        "x", "y"
    )
)
    
show(plot.draw())

That's nice, but all the players look the same, which isn't very helpful. The good news is that it's pretty easy to make things clearer. Setting the appropriate `Aesthetics` automatically takes care of the team colors and the ball, adding jersey numbers to the position markers is a simple keyword argument, and we can even add labels on mouseover with the `Hover` layer:

In [5]:
plot = (
    PTPlot(data=player_tracking_data[player_tracking_data["event"] == "pass_forward"], pixel_height=400) 
    + Field() 
    + Positions(
        "x", "y", number="jerseyNumber",
        name="positions" # The name is used to map the Hover label to a layer
    )
    + Aesthetics(team_ball_mapping="teamAbbr", home_away_mapping="homeTeamFlag == 1", ball_identifier="ball")
    + Hover([("name", "@displayName")], "positions", ["displayName"])
)
    
show(plot.draw())

Much better. You can also use the Bokeh controls on the top right of the chart to do things like pan and save a static version of the plot.

in addition to making plots of player positions, `ptplot` can also generate tracks of player positions over the course of the play. The code in the next cell generates those tracks for the entire play:

In [6]:
plot = (
    PTPlot(data=player_tracking_data, pixel_height=400) 
    + Field() 
    + Tracks("x", "y", "displayName", line_width=2)
    + Aesthetics(team_ball_mapping="teamAbbr", home_away_mapping="homeTeamFlag == 1", ball_identifier="ball")
)
    
show(plot.draw())

Plotting both tracks and positions is as simple as adding both layers to the `PTPlot` object:

In [7]:
# Just look between the snap and the pass, to make the tracks clearer
snap_frame = player_tracking_data[player_tracking_data["event"] == "ball_snap"]["frame"].unique()[0]
pass_forward_frame = player_tracking_data[player_tracking_data["event"] == "pass_forward"]["frame"].unique()[0]

plot = (
    PTPlot(
        data=player_tracking_data[player_tracking_data.frame.between(snap_frame, pass_forward_frame)], 
        pixel_height=400
    ) 
    + Field() 
    + Tracks("x", "y", "displayName", line_width=2)
    + Positions(
        "x", "y", number="jerseyNumber",
        frame_filter = f"frame == {pass_forward_frame}", # only show one frame for the positions
        name="positions" # The name is used to map the Hover label to a layer
    )
    + Aesthetics(team_ball_mapping="teamAbbr", home_away_mapping="homeTeamFlag == 1", ball_identifier="ball")
    + Hover([("name", "@displayName")], "positions", ["displayName"])
)
    
show(plot.draw())